In [192]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pycountry


# Languages

In [193]:
# url of a website containg most common languages per country
language_url = "https://www.infoplease.com/world/countries/languages-spoken-in-each-country-of-the-world"

In [194]:
# get html page
page = requests.get(language_url)

# transform to soup object
soup = BeautifulSoup(page.content, 'html.parser')
    
# find the table with all information (right, top of wikipedia article)
table = soup.find('table')

# get all the rows of the table, discard the header line
rows = table.find_all("tr")[1:]

In [195]:

country_languages = {}

# regular expressions
find_country_regex = r">.*<"
remove_bracket_regex = r'\([^\]]*\)'

# extract countries and corresponding languages
for row in rows:
    # get the country names
    country = str(re.findall(find_country_regex, str(row.findChild("a")))[0][1:-1])
    # remove hexadecimals
    country = country.encode('ascii', errors='ignore').decode("UTF-8")
    # remove brackets
    country = re.sub(remove_bracket_regex, "", country)
    # replace st. by sant
    country = country.replace("St.", "Saint")
    #country = country.decode('utf8').encode('ascii', errors='ignore')
    # contains the whole row of one country
    temp_list = str(row.findAll('p')[1])[3:-4].split(",")
    # extract only the language names
    languages = []
    for language in temp_list:
        # only get 3 languages
        if len(languages) < 3:
            languages.append(re.findall(r"([^\s]+)", language)[0])
    # add to dictionary
    country_languages[country] = languages



In [196]:
# fix wrong words
remove_words = ["other", "and", "widely", "tribal", "24", "60", "declining", "Qazaq)", 
                "multilingual", "indigenous", "which", "small", "unspecified", "but", "universal",
               "regular", "120", "the", "only", "one", "taught", "used", "local", "some", "numerous", 
                "a", "Standard", "based"]

for country, language in country_languages.items():
    for illegal_word in remove_words:
        if illegal_word in country_languages[country]:
            country_languages[country].remove(illegal_word)
################################################################################################
# fix country names for pycountry
replace_country_names_dic = {"Congo, Democratic Republic of the":"Congo, The Democratic Republic of the",     
                        "Congo, Republic of": "Congo",
                        "Cape Verde":"Cabo Verde",
                        "Palestinian State ":"Palestine, State of",
                        "East Timor":"Timor-Leste",
                        "Korea, North":"Korea, Democratic People's Republic of",
                        "Korea, South":"Korea, Republic of",
                        "Laos":"Lao People's Democratic Republic",
                        "So Tom and Prncipe":"Sao Tome and Principe",
                        "Cte d'Ivoire":"Côte d'Ivoire"}
# since we cannot change the size of the dictionary withing a loop, save the ones that are supposed
# to be deleted and their replacements
to_delete = []
to_add = {}
# depending on replace_country_names_dic, look at what to replace and what to replace with
for country, language_list in country_languages.items():
    if country in replace_country_names_dic.keys():
        to_add[replace_country_names_dic[country]] = language_list
        to_delete.append(country)
        
# delete old names
for country in to_delete:
    del country_languages[country]
    
# add new ones
country_languages = {**country_languages, **to_add}

################################################################################################
# manual fixes
country_languages["China"].extend(["Mandarin", "Chinese"])
country_languages["Western Sahara"].extend(["Arabic"])
country_languages["Philippines"].extend(["English"])

In [191]:
# for creating a df
country_name_list = []
country_alpha_2_list = []
country_alpha_3_list = []
languages_list = []

for country, language_list in country_languages.items():
    # use the pycountry package to obtain the common names, and the alpha2 and alpha3 abbreviations
    pycountry_object = pycountry.countries.search_fuzzy(country)[0]
    country_name_list.append(pycountry_object.name)
    country_alpha_2_list.append(pycountry_object.alpha_2)
    country_alpha_3_list.append(pycountry_object.alpha_3)
    languages_list.append(language_list)


In [197]:
country_lang_df = df = pd.DataFrame(list(zip(country_alpha_2_list, country_alpha_3_list, 
                                             country_name_list, languages_list)), 
                                   columns =["name", "alpha_2", "alpha_3", "languages"]) 

In [198]:
country_lang_df

,name,alpha_2,alpha_3,languages
0,AF,AFG,Afghanistan,"[Afghan, Pashto, Uzbek]"
1,AL,ALB,Albania,"[Albanian, Greek]"
2,DZ,DZA,Algeria,"[Arabic, French, Berber]"
3,AD,AND,Andorra,"[Catalan, French, Castilian]"
4,AO,AGO,Angola,"[Portuguese, Umbundu, Kikongo]"
...,...,...,...,...
193,KP,PRK,"Korea, Democratic People's Republic of",[Korean]
194,KR,KOR,"Korea, Republic of","[Korean, English, junior]"
195,LA,LAO,Lao People's Democratic Republic,"[Lao, French, English]"
196,PS,PSE,"Palestine, State of","[Palauan, English]"
